<a href="https://colab.research.google.com/github/soham7707/DeepSectorAI/blob/main/BiLSTM_GARCH_BETA__Bullish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install arch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.3/978.3 kB 17.2 MB/s eta 0:00:00


In [ ]:
pip install yfinance pandas numpy matplotlib


In [ ]:
# Define your sectors and representative stocks
sectors = {
    'IT': ['TCS.NS', 'INFY.NS', 'HCLTECH.NS', 'WIPRO.NS'],
    'Banking': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
    'Pharma': ['SUNPHARMA.NS', 'CIPLA.NS', 'DRREDDY.NS'],
    'Auto': ['TATAMOTORS.NS', 'M&M.NS', 'MARUTI.NS'],
    'FMCG': ['HINDUNILVR.NS', 'ITC.NS', 'DABUR.NS']
}


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime

start_date = "2020-04-01"    #Bullish period
end_date = "2021-10-01"

def fetch_sector_data(sectors):
    all_data = {}
    for sector, stocks in sectors.items():
        print(f"Fetching data for {sector} sector...")
        data = yf.download(stocks, start=start_date, end=end_date)['Close']
        all_data[sector] = data
    return all_data

sector_data = fetch_sector_data(sectors)


Fetching data for IT sector...


/tmp/ipython-input-1433047203.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']
[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-1433047203.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Banking sector...


[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-1433047203.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Pharma sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-1433047203.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Auto sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-1433047203.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for FMCG sector...


[*********************100%***********************]  3 of 3 completed


handling missing data


In [ ]:
for sector in sector_data:
    df = sector_data[sector]
    df = df.ffill().bfill()  # Forward and backward fill
    sector_data[sector] = df


Calculating Log Returns

In [ ]:
import numpy as np

returns_data = {}

for sector, df in sector_data.items():
    returns = np.log(df / df.shift(1)).dropna()
    returns_data[sector] = returns


In [ ]:
for sector, df in returns_data.items():
    df.to_csv(f"{sector}_returns.csv")


In [ ]:
sector_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2022-01-03,693.454529,1449.202515,741.773926,1817.761841
2022-01-04,706.201416,1457.689941,749.679565,1845.808594
2022-01-05,723.877563,1492.307129,764.423828,1915.103638
2022-01-06,727.263428,1468.370605,761.513794,1884.964600
2022-01-07,727.562134,1478.670044,769.467957,1897.269287
...,...,...,...,...
2022-06-23,626.733093,1288.059692,678.189148,1677.926392
2022-06-24,631.363892,1306.052002,692.060425,1692.173950
2022-06-27,634.450867,1308.415527,695.843445,1685.348999


In [ ]:
returns_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2022-01-04,0.018215,0.005840,0.010601,0.015311
2022-01-05,0.024722,0.023470,0.019477,0.036854
2022-01-06,0.004666,-0.016170,-0.003814,-0.015863
2022-01-07,0.000411,0.006990,0.010391,0.006507
2022-01-10,0.016561,0.005531,0.021821,0.022689
...,...,...,...,...
2022-06-23,0.004060,0.003677,0.018113,0.008587
2022-06-24,0.007362,0.013872,0.020247,0.008455
2022-06-27,0.004877,0.001808,0.005451,-0.004041


In [ ]:
pip install arch


In [ ]:
from arch import arch_model
import pandas as pd
import numpy as np


In [ ]:
def estimate_garch_volatility(returns_series):
    # Fit GARCH(1,1) model to the returns series
    model = arch_model(returns_series, vol='GARCH', p=1, q=1)
    res = model.fit(disp='off')
    # Return the estimated conditional volatility (σ_t)
    return res.conditional_volatility


In [ ]:
garch_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Estimating GARCH volatility for {sector} sector...")
    sector_vols = pd.DataFrame(index=returns_df.index)

    for stock in returns_df.columns:
        try:
            vol = estimate_garch_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error processing {stock}: {e}")

    garch_volatility_data[sector] = sector_vols


Estimating GARCH volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003374. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003659. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarn

Estimating GARCH volatility for Banking sector...
Estimating GARCH volatility for Pharma sector...
Estimating GARCH volatility for Auto sector...
Estimating GARCH volatility for FMCG sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002738. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003046. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarn

In [ ]:
print(garch_volatility_data['FMCG'].tail())


            DABUR.NS  HINDUNILVR.NS    ITC.NS
Date                                         
2022-06-23  0.017313       0.017924  0.014786
2022-06-24  0.016516       0.017976  0.014786
2022-06-27  0.015484       0.018025  0.014786
2022-06-28  0.015255       0.017976  0.014786
2022-06-29  0.014115       0.017920  0.014786


LSTM

In [ ]:
pip install numpy pandas scikit-learn tensorflow


In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def prepare_lstm_data(series, lookback=20):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(series.values.reshape(-1, 1))

    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i])
        y.append(scaled_data[i])

    X = np.array(X)
    y = np.array(y)
    return X, y, scaler


In [ ]:
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import LSTM, Dense, Bidirectional


def build_bilstm_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=50, return_sequences=False), input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model




In [ ]:
def predict_bilstm_volatility(returns_series, lookback=20):
    X, y, scaler = prepare_lstm_data(returns_series, lookback)
    model = build_bilstm_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=10, batch_size=16, verbose=0)

    y_pred_scaled = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()

    predicted_volatility = pd.Series(y_pred).rolling(window=lookback).std()
    predicted_volatility.index = returns_series.index[lookback:]
    return predicted_volatility


In [ ]:
bilstm_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Predicting BiLSTM volatility for {sector} sector...")
    sector_vols = pd.DataFrame()

    for stock in returns_df.columns:
        try:
            vol = predict_bilstm_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error with {stock}: {e}")

    bilstm_volatility_data[sector] = sector_vols


Predicting BiLSTM volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 771ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 237ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step


In [ ]:
print(bilstm_volatility_data['IT'].tail())


            HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2022-06-23    0.001517  0.001793  0.001615  0.001846
2022-06-24    0.001464  0.002068  0.001658  0.001660
2022-06-27    0.001375  0.002040  0.001650  0.001501
2022-06-28    0.001374  0.001978  0.001632  0.001513
2022-06-29    0.001358  0.001978  0.001633  0.001506


In [ ]:
bilstm_volatility_data['IT'].dropna

<bound method DataFrame.dropna of             HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2022-02-02         NaN       NaN       NaN       NaN
2022-02-03         NaN       NaN       NaN       NaN
2022-02-04         NaN       NaN       NaN       NaN
2022-02-07         NaN       NaN       NaN       NaN
2022-02-08         NaN       NaN       NaN       NaN
...                ...       ...       ...       ...
2022-06-23    0.001517  0.001793  0.001615  0.001846
2022-06-24    0.001464  0.002068  0.001658  0.001660
2022-06-27    0.001375  0.002040  0.001650  0.001501
2022-06-28    0.001374  0.001978  0.001632  0.001513
2022-06-29    0.001358  0.001978  0.001633  0.001506

[101 rows x 4 columns]>

In [ ]:
bilstm_volatility_data['IT'].head()

,HCLTECH.NS,INFY.NS,TCS.NS,WIPRO.NS
Date,,,,
2022-02-02,NaN,NaN,NaN,NaN
2022-02-03,NaN,NaN,NaN,NaN
2022-02-04,NaN,NaN,NaN,NaN
2022-02-07,NaN,NaN,NaN,NaN
2022-02-08,NaN,NaN,NaN,NaN


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

# Download NIFTY-50 index (symbol: ^NSEI)
nifty = yf.download("^NSEI", start="2015-01-01")['Close']
nifty_returns = np.log(nifty / nifty.shift(1)).dropna()
nifty_returns_df = pd.DataFrame(nifty_returns)
nifty_returns_df.columns = ['Market']


/tmp/ipython-input-2331807048.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  nifty = yf.download("^NSEI", start="2015-01-01")['Close']
[*********************100%***********************]  1 of 1 completed


In [ ]:
import statsmodels.api as sm

def compute_beta(stock_returns, market_returns_df):
    aligned = pd.concat([stock_returns, market_returns_df], axis=1).dropna()
    aligned.columns = ['Stock', 'Market']

    X = sm.add_constant(aligned['Market'])
    y = aligned['Stock']
    model = sm.OLS(y, X).fit()
    return model.params['Market']  # This is the beta



In [ ]:
beta_values = {}

for sector, returns_df in returns_data.items():
    print(f"Computing Beta for {sector} sector...")
    sector_betas = {}

    for stock in returns_df.columns:
        try:
            beta = compute_beta(returns_df[stock], nifty_returns_df)
            sector_betas[stock] = beta
        except Exception as e:
            print(f"Error with {stock}: {e}")

    beta_values[sector] = sector_betas


Computing Beta for IT sector...
Computing Beta for Banking sector...
Computing Beta for Pharma sector...
Computing Beta for Auto sector...
Computing Beta for FMCG sector...


In [ ]:
beta_df = pd.DataFrame(beta_values).T  # Transpose to have stocks as columns
beta_df = beta_df.fillna(0)
print(beta_df)


         HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS  AXISBANK.NS  HDFCBANK.NS  \
IT         0.935741  1.029992  0.881972  1.100391     0.000000     0.000000   
Banking    0.000000  0.000000  0.000000  0.000000     1.025735     1.076282   
Pharma     0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   
Auto       0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   
FMCG       0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   

         ICICIBANK.NS  KOTAKBANK.NS  CIPLA.NS  DRREDDY.NS  SUNPHARMA.NS  \
IT            0.00000      0.000000  0.000000    0.000000      0.000000   
Banking       1.00477      1.003439  0.000000    0.000000      0.000000   
Pharma        0.00000      0.000000  0.378681    0.732037      0.635612   
Auto          0.00000      0.000000  0.000000    0.000000      0.000000   
FMCG          0.00000      0.000000  0.000000    0.000000      0.000000   

           M&M.NS  MARUTI.NS  TATAMOTORS.NS  DABUR.NS  HINDUNILVR.NS   ITC

Computing Sharpe Ratio

In [ ]:
sharpe_ratios = {}

for sector, df in returns_data.items():
    sector_sharpes = {}
    for stock in df.columns:
        r = df[stock].dropna()
        mean_return = r.mean()
        std_return = r.std()
        sharpe = mean_return / std_return if std_return != 0 else 0
        sector_sharpes[stock] = sharpe
    sharpe_ratios[sector] = sector_sharpes


In [ ]:
features = []

for sector in returns_data:
    for stock in returns_data[sector].columns:
        try:
            beta = beta_values[sector][stock]
            garch_mean = garch_volatility_data[sector][stock].mean()
            bilstm_mean = bilstm_volatility_data[sector][stock].mean()
            sharpe = sharpe_ratios[sector][stock]
            features.append([sector, stock, beta, garch_mean, bilstm_mean, sharpe])
        except Exception as e:
            print(f"Skipping {stock} due to missing data: {e}")

features_df = pd.DataFrame(features, columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol', 'BiLSTM_Vol', 'Sharpe'])


In [ ]:
features_df = pd.DataFrame(features, columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol', 'BiLSTM_Vol', 'Sharpe'])


In [ ]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'BiLSTM_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)


Learned Weights: [-0.20848735  7.04842285 28.61844348]
Intercept: -0.03125913011902867


In [ ]:
features_df['Risk_Score'] = X @ weights


In [ ]:
for sector in features_df['Sector'].unique():
    print(f"\nTop 3 safest stocks in {sector}:")
    print(features_df[features_df['Sector'] == sector].sort_values('Risk_Score').head(3)[['Stock', 'Risk_Score']])



Top 3 safest stocks in IT:
        Stock  Risk_Score
2      TCS.NS   -0.029644
0  HCLTECH.NS   -0.027325
3    WIPRO.NS   -0.024046

Top 3 safest stocks in Banking:
          Stock  Risk_Score
6  ICICIBANK.NS   -0.036920
7  KOTAKBANK.NS   -0.035004
5   HDFCBANK.NS   -0.018225

Top 3 safest stocks in Pharma:
           Stock  Risk_Score
10  SUNPHARMA.NS    0.018473
9     DRREDDY.NS    0.040114
8       CIPLA.NS    0.065526

Top 3 safest stocks in Auto:
            Stock  Risk_Score
13  TATAMOTORS.NS   -0.065067
11         M&M.NS    0.003581
12      MARUTI.NS    0.015259

Top 3 safest stocks in FMCG:
            Stock  Risk_Score
15  HINDUNILVR.NS    0.013747
16         ITC.NS    0.017058
14       DABUR.NS    0.033734


BiLSTM AND GARCH ONLY

In [ ]:
from sklearn.linear_model import LinearRegression

X_new = features_df[['GARCH_Vol', 'BiLSTM_Vol']]
y_new = features_df['Sharpe']

model_new = LinearRegression().fit(X_new, y_new)
weights_new = model_new.coef_
intercept_new = model_new.intercept_

print("New Weights (GARCH, LSTM):", weights_new)


New Weights (GARCH, LSTM): [-9.01528981 20.48077024]


In [ ]:
features_df['Risk_Score_BiLSTM_GARCH'] = X_new @ weights_new


In [ ]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (GARCH + BiLSTM only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_BiLSTM_GARCH')
          .head(3)[['Stock', 'Risk_Score_BiLSTM_GARCH']])



Safest stocks in IT (GARCH + BiLSTM only):
        Stock  Risk_Score_BiLSTM_GARCH
3    WIPRO.NS                -0.136745
0  HCLTECH.NS                -0.136397
1     INFY.NS                -0.119824

Safest stocks in Banking (GARCH + BiLSTM only):
          Stock  Risk_Score_BiLSTM_GARCH
7  KOTAKBANK.NS                -0.127187
5   HDFCBANK.NS                -0.122011
4   AXISBANK.NS                -0.117436

Safest stocks in Pharma (GARCH + BiLSTM only):
           Stock  Risk_Score_BiLSTM_GARCH
10  SUNPHARMA.NS                -0.116368
8       CIPLA.NS                -0.113872
9     DRREDDY.NS                -0.106393

Safest stocks in Auto (GARCH + BiLSTM only):
            Stock  Risk_Score_BiLSTM_GARCH
13  TATAMOTORS.NS                -0.185562
12      MARUTI.NS                -0.140574
11         M&M.NS                -0.124145

Safest stocks in FMCG (GARCH + BiLSTM only):
            Stock  Risk_Score_BiLSTM_GARCH
15  HINDUNILVR.NS                -0.120161
14       DABUR.NS    

BiLSTM AND BETA ONLY

In [ ]:
from sklearn.linear_model import LinearRegression

X_lb = features_df[['Beta', 'BiLSTM_Vol']]
y_lb = features_df['Sharpe']

model_lb = LinearRegression().fit(X_lb, y_lb)
weights_lb = model_lb.coef_
intercept_lb = model_lb.intercept_

print("Weights: Beta =", weights_lb[0], ", LSTM =", weights_lb[1])
print("Intercept:", intercept_lb)


Weights: Beta = -0.15157589358614723 , LSTM = 35.525779298064606
Intercept: 0.03184606334752592


In [ ]:
features_df['Risk_Score_BiLSTM_Beta'] = X_lb @ weights_lb


In [ ]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (BiLSTM + Beta only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_BiLSTM_Beta')
          .head(3)[['Stock', 'Risk_Score_BiLSTM_Beta']])



Safest stocks in IT (BiLSTM + Beta only):
        Stock  Risk_Score_BiLSTM_Beta
0  HCLTECH.NS               -0.093180
3    WIPRO.NS               -0.088420
2      TCS.NS               -0.082938

Safest stocks in Banking (BiLSTM + Beta only):
          Stock  Risk_Score_BiLSTM_Beta
7  KOTAKBANK.NS               -0.092588
6  ICICIBANK.NS               -0.081370
5   HDFCBANK.NS               -0.074964

Safest stocks in Pharma (BiLSTM + Beta only):
           Stock  Risk_Score_BiLSTM_Beta
10  SUNPHARMA.NS               -0.048575
9     DRREDDY.NS               -0.023756
8       CIPLA.NS               -0.013263

Safest stocks in Auto (BiLSTM + Beta only):
            Stock  Risk_Score_BiLSTM_Beta
13  TATAMOTORS.NS               -0.144876
12      MARUTI.NS               -0.061366
11         M&M.NS               -0.059481

Safest stocks in FMCG (BiLSTM + Beta only):
            Stock  Risk_Score_BiLSTM_Beta
15  HINDUNILVR.NS               -0.052389
16         ITC.NS               -0.040917
14

GARCH AND BETA ONLY

In [ ]:
from sklearn.linear_model import LinearRegression

X_gb = features_df[['Beta', 'GARCH_Vol']]
y_gb = features_df['Sharpe']

model_gb = LinearRegression().fit(X_gb, y_gb)
weights_gb = model_gb.coef_
intercept_gb = model_gb.intercept_

print("Weights: Beta =", weights_gb[0], ", GARCH =", weights_gb[1])
print("Intercept:", intercept_gb)


Weights: Beta = -0.19736679570229282 , GARCH = 9.712722245565462
Intercept: -0.031630665628267246


In [ ]:
features_df['Risk_Score_GARCH_Beta'] = X_gb @ weights_gb


In [ ]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (GARCH + Beta only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GARCH_Beta')
          .head(3)[['Stock', 'Risk_Score_GARCH_Beta']])



Safest stocks in IT (GARCH + Beta only):
      Stock  Risk_Score_GARCH_Beta
3  WIPRO.NS              -0.021179
2    TCS.NS              -0.017868
1   INFY.NS              -0.017704

Safest stocks in Banking (GARCH + Beta only):
          Stock  Risk_Score_GARCH_Beta
6  ICICIBANK.NS              -0.039255
5   HDFCBANK.NS              -0.026206
7  KOTAKBANK.NS              -0.024058

Safest stocks in Pharma (GARCH + Beta only):
           Stock  Risk_Score_GARCH_Beta
9     DRREDDY.NS               0.024306
10  SUNPHARMA.NS               0.029590
8       CIPLA.NS               0.075355

Safest stocks in Auto (GARCH + Beta only):
            Stock  Risk_Score_GARCH_Beta
13  TATAMOTORS.NS              -0.050860
11         M&M.NS              -0.010359
12      MARUTI.NS               0.013030

Safest stocks in FMCG (GARCH + Beta only):
            Stock  Risk_Score_GARCH_Beta
15  HINDUNILVR.NS               0.014730
16         ITC.NS               0.021400
14       DABUR.NS               0.

In [ ]:
features_df[['Stock', 'Risk_Score', 'Risk_Score_BiLSTM_GARCH', 'Risk_Score_BiLSTM_Beta', 'Risk_Score_GARCH_Beta']].head()


,Stock,Risk_Score,Risk_Score_BiLSTM_GARCH,Risk_Score_BiLSTM_Beta,Risk_Score_GARCH_Beta
0,HCLTECH.NS,-0.027325,-0.136397,-0.093180,-0.007516
1,INFY.NS,-0.006800,-0.119824,-0.065173,-0.017704
2,TCS.NS,-0.029644,-0.115731,-0.082938,-0.017868
3,WIPRO.NS,-0.024046,-0.136745,-0.088420,-0.021179
4,AXISBANK.NS,-0.000438,-0.117436,-0.058685,-0.015807


Comparison per sector

In [ ]:
def compare_sector_models(sector_name, top_n=5):
    df = features_df[features_df['Sector'] == sector_name].copy()

    models = ['Risk_Score', 'Risk_Score_BiLSTM_GARCH', 'Risk_Score_BiLSTM_Beta', 'Risk_Score_GARCH_Beta']

    print(f"\n📊 Sector: {sector_name}")
    for model in models:
        print(f"\n🔹 Top {top_n} safest stocks using {model}:")
        display(df.sort_values(model).head(top_n)[['Stock', model]])


In [ ]:
compare_sector_models("IT", top_n=3)
compare_sector_models("Banking", top_n=3)
compare_sector_models("Pharma", top_n=3)



📊 Sector: IT

🔹 Top 3 safest stocks using Risk_Score:


,Stock,Risk_Score
2,TCS.NS,-0.029644
0,HCLTECH.NS,-0.027325
3,WIPRO.NS,-0.024046



🔹 Top 3 safest stocks using Risk_Score_BiLSTM_GARCH:


,Stock,Risk_Score_BiLSTM_GARCH
3,WIPRO.NS,-0.136745
0,HCLTECH.NS,-0.136397
1,INFY.NS,-0.119824



🔹 Top 3 safest stocks using Risk_Score_BiLSTM_Beta:


...

,Stock,Risk_Score_BiLSTM_GARCH
10,SUNPHARMA.NS,-0.116368
8,CIPLA.NS,-0.113872
9,DRREDDY.NS,-0.106393



🔹 Top 3 safest stocks using Risk_Score_BiLSTM_Beta:


,Stock,Risk_Score_BiLSTM_Beta
10,SUNPHARMA.NS,-0.048575
9,DRREDDY.NS,-0.023756
8,CIPLA.NS,-0.013263



🔹 Top 3 safest stocks using Risk_Score_GARCH_Beta:


,Stock,Risk_Score_GARCH_Beta
9,DRREDDY.NS,0.024306
10,SUNPHARMA.NS,0.029590
8,CIPLA.NS,0.075355


Comparison table

In [ ]:
def sector_ranking_table(sector_name):
    df = features_df[features_df['Sector'] == sector_name].copy()
    ranking_df = df[['Stock']].copy()

    for col in ['Risk_Score', 'Risk_Score_BiLSTM_GARCH', 'Risk_Score_BiLSTM_Beta', 'Risk_Score_GARCH_Beta']:
        ranking_df[col + '_Rank'] = df[col].rank(method='min')

    return ranking_df.sort_values('Risk_Score_Rank')


In [ ]:
sector_ranking_table("FMCG")


,Stock,Risk_Score_Rank,Risk_Score_BiLSTM_GARCH_Rank,Risk_Score_BiLSTM_Beta_Rank,Risk_Score_GARCH_Beta_Rank
15,HINDUNILVR.NS,1.0,1.0,1.0,1.0
16,ITC.NS,2.0,3.0,2.0,2.0
14,DABUR.NS,3.0,2.0,3.0,3.0
